## **Data downloading**

Colab to automatically download genes from NCBI database.

> We wanted to use partitioning for this task but the problem is data:

*   18S: whole genome downloading (too big files) 33/215
*   28S: whole genome downloading (too big files) 88/215
*   COI: only one which is presented for all species
*   ITS1: 16/215
*   Cytb: 62/215 mitochondrion, complete genome

---

**No partitioning then** 😞

In [ ]:
%%capture
!pip install Bio

In [ ]:
import pandas as pd
from Bio import Entrez, SeqIO
Entrez.email = "email"  # Замените на ваш email
from urllib.error import HTTPError
import time
import os
from collections import defaultdict
from Bio.SeqRecord import SeqRecord

In [ ]:
# Чтение таблицы
df = pd.read_csv('/content/species_list_PG.csv')

# Замена "_" на пробелы в названиях видов
df['Species'] = df['Species'].str.replace('_', ' ')

# Просмотр первых строк
df.head()

,Species,Genbank_assembly_accession,Unnamed: 2
0,Glyphotaelius pellucidus,GCA_936435175.1,NaN
1,Hydropsyche tenuis,GCA_009617725.1,NaN
2,Limnephilus lunatus,GCA_917563855.2,NaN
3,Limnephilus marmoratus,GCA_917880885.1,NaN
4,Limnephilus rhombicus,GCA_929108145.1,NaN


## **COI**

In [ ]:
# Функция для поиска и получения последовательностей COI
def fetch_coi_sequences(species, max_retries=3):
    search_term = f'"{species}"[Organism] AND COI[Gene]'
    for attempt in range(max_retries):
        try:
            handle = Entrez.esearch(db="nucleotide", term=search_term, retmode="xml", retmax=1)
            record = Entrez.read(handle)
            handle.close()
            id_list = record['IdList']
            if id_list:
                # Получение последовательности COI
                fetch_handle = Entrez.efetch(db="nucleotide", id=id_list[0], rettype="fasta", retmode="text")
                seq_record = SeqIO.read(fetch_handle, "fasta")
                fetch_handle.close()
                return seq_record
            else:
                print(f"Последовательность COI не найдена для вида {species}.")
                return None
        except Exception as e:
            print(f"Ошибка при запросе для {species} - COI: {e}")
            return None

# Инициализация списков для записей
COI_records = []

# Путь для сохранения последовательностей
if not os.path.exists('sequences'):
    os.makedirs('sequences')

# Словари для отслеживания успешных и неуспешных загрузок
success_dict = defaultdict(list)
fail_dict = defaultdict(list)

# Обработка каждого вида
for index, row in df.iterrows():
    species = row['Species']

    # Загрузка COI
    print(f"Загрузка COI для {species}...")
    coi_seq = fetch_coi_sequences(species)
    if coi_seq:
        COI_records.append(coi_seq)
        success_dict['COI'].append(species)
    else:
        fail_dict['COI'].append(species)

    # Задержка для соблюдения правил NCBI
    time.sleep(0.4)  # примерно 2.5 запроса в секунду

# Сохранение в FASTA файл
SeqIO.write(COI_records, "sequences/COI_sequences.fasta", "fasta")

# Вывод статистики
print("\nЗагрузка COI завершена.")
print(f"COI: успешно загружено {len(success_dict['COI'])} видов, не найдено {len(fail_dict['COI'])} видов.")

# Сохранение списка неуспешных загрузок для дальнейшей проверки
with open("sequences/failed_COI.txt", "w") as f:
    for species in fail_dict['COI']:
        f.write(f"{species}\n")


Загрузка COI для Glyphotaelius pellucidus...
Загрузка COI для Hydropsyche tenuis...
Загрузка COI для Limnephilus lunatus...
Загрузка COI для Limnephilus marmoratus...
Загрузка COI для Limnephilus rhombicus...
Загрузка COI для Abrostola tripartita...
Загрузка COI для Acentria ephemerella...
Загрузка COI для Acleris emargana...
Загрузка COI для Acleris sparsana...
Загрузка COI для Acrobasis suavella...
Загрузка COI для Acronicta aceris...
Загрузка COI для Agonopterix arenella...
Загрузка COI для Agonopterix subpropinquella...
Загрузка COI для Agriopis aurantiaria...
Загрузка COI для Agriopis marginaria...
Загрузка COI для Agriphila geniculea...
Загрузка COI для Agriphila tristella...
Загрузка COI для Agrochola circellaris...
Загрузка COI для Agrochola macilenta...
Загрузка COI для Agrotis puta...
Загрузка COI для Allophyes oxyacanthae...
Загрузка COI для Amphipyra berbera...
Загрузка COI для Amphipyra tragopoginis...
Загрузка COI для Anthocharis cardamines...
Загрузка COI для Apamea mono

## **28S, ITS1, 18S, Cytb**

The same code as used for COI.

## **Check what is present**

In [ ]:
# Список генов и их соответствующие файлы
genes = {
    "ITS1": "/content/sequences/ITS1_sequences.fasta",
    "18S": "/content/sequences/18S_sequences.fasta",
    "28S": "/content/sequences/28S_sequences.fasta",
    "COI": "/content/sequences/COI_sequences.fasta"
}

# Словарь для хранения информации о наличии последовательностей
species_dict = {}

# Функция для извлечения названий видов из файла FASTA
def extract_species_from_fasta(fasta_file):
    species_list = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Предполагается, что название вида можно извлечь из заголовка записи
        species_name = record.description.split()[1:3]  # берем второе и третье слово (род и вид)
        species_name = " ".join(species_name)  # объединяем их в название вида
        species_list.append(species_name)
    return species_list

# Проходим по каждому файлу и добавляем информацию в species_dict
for gene, fasta_file in genes.items():
    if os.path.exists(fasta_file):
        species_in_file = extract_species_from_fasta(fasta_file)
        for species in species_in_file:
            if species not in species_dict:
                species_dict[species] = {}
            species_dict[species][gene] = True

# Преобразуем в DataFrame для удобного представления
df = pd.DataFrame.from_dict(species_dict, orient="index")

# Заполним пропущенные значения (где нет последовательностей) как False
df.fillna(False, inplace=True)

# Добавляем столбец с подсчётом количества True для каждого вида
df['Total True'] = df.sum(axis=1)

# Сохраняем результат в файл
df.to_csv("gene_presence_table_with_count.csv")

<ipython-input-10-a28253824d8a>:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(False, inplace=True)


In [ ]:
df = pd.read_csv('/content/gene_presence_table_with_count.csv')
df.head()

,Unnamed: 0,ITS1,28S,COI,18S,Total True
0,Bombyx mori,True,True,True,False,3
1,Dendrolimus punctatus,True,True,True,True,4
2,Helicoverpa zea,True,True,True,True,4
3,Hyles euphorbiae,True,True,True,False,3
4,Nymphalis io,True,True,True,True,4


## **Species list to rename tree**

In [ ]:
from Bio import SeqIO
import json

# Функция для извлечения названия вида из заголовка
def extract_species_name(header):
    # Название вида после первого пробела
    parts = header.split(' ')
    if len(parts) >= 2:
        species = ' '.join(parts[1:3])
        return species
    else:
        return None

# Создание словаря с GenBank accession -> Название вида
def create_species_dict(input_fasta, output_dict_file):
    species_dict = {}

    for record in SeqIO.parse(input_fasta, "fasta"):
        accession_number = record.id
        species_name = extract_species_name(record.description)

        if species_name:
            species_dict[accession_number] = species_name

    # Сохранение словаря в файл
    with open(output_dict_file, "w") as outfile:
        json.dump(species_dict, outfile, indent=4)

    print(f"Словарь сохранен в {output_dict_file}")

# Применение функции для COI
create_species_dict("sequences/COI_sequences.fasta", "COI_species_dict.json")

Словарь сохранен в COI_species_dict.json


## **Rename the tree**

In [1]:
import json

# Load the JSON file
with open("COI_species_dict.json", "r") as file:
    data = json.load(file)

# Replace spaces with underscores in species names
updated_data = {key: value.replace(" ", "_") for key, value in data.items()}

# Save the updated JSON back to a file
with open("COI_species_dict_updated.json", "w") as file:
    json.dump(updated_data, file, indent=4)

print("Spaces replaced with underscores successfully!")


Spaces replaced with underscores successfully!


In [2]:
import json

# Загрузка словаря
with open("COI_species_dict_updated.json") as dict_file:
    species_dict = json.load(dict_file)

# Функция для переименования узлов дерева
def rename_tree_nodes(input_tree_file, output_tree_file):
    with open(input_tree_file, "r") as infile:
        tree_data = infile.read()

    # Замена GenBank accession на названия видов
    for accession, species in species_dict.items():
        tree_data = tree_data.replace(accession, species)

    # Сохранение измененного дерева
    with open(output_tree_file, "w") as outfile:
        outfile.write(tree_data)

    print(f"Дерево переименовано и сохранено в {output_tree_file}")

# Пример применения функции для дерева
rename_tree_nodes("/content/COI_trimmed.fasta.treefile", "renamed_tree_file.nwk")

Дерево переименовано и сохранено в renamed_tree_file.nwk
